## Testing MCP server (local-fake remote) with OpenAI 

In [ ]:
import requests
from openai import OpenAI
import os 
from dotenv import load_dotenv
import json
load_dotenv()

def get_token():
    token_url = 'https://oauth.piste.gouv.fr/api/oauth/token'
    #inject cred 
    token_data = {
    'grant_type': 'client_credentials',
    'client_id': client_id_prod,
    'client_secret': client_secret_prod,
    'scope': 'openid'}
    response = requests.post(token_url, data=token_data)
    response.raise_for_status()  # vérif  erreurs
    # récup  jeton
    token_info = response.json()
    access_token = token_info['access_token']
    return access_token

#loading tokens
github_barer=os.getenv('github_barer')
client_id_prod=os.getenv('client_id_prod')
client_secret_prod=os.getenv('client_secret_prod')

## loading MCP file configuration
with open("/Users/oussa/Desktop/Github_perso/chatbot_mcp/mcp_server_config/mcp_server_config_openai.json", "r") as file:
    mcp_server_config_data = json.load(file)
                
## inject github token 
for s in mcp_server_config_data:
    if s["server_label"] == "github":
        s["headers"]["Authorization"] = f"Bearer {github_barer}"
   
## get LegiFrance token 
token=get_token()

resp = OpenAI().responses.create(
    #model="gpt-5-nano-2025-08-07",
    model='gpt-5-mini-2025-08-07',
    #model='gpt-5-2025-08-07',
    tools= mcp_server_config_data,
#input="clone ce repos https://github.com/Bendrox/MCP_chatbot.git et fait moi une proposition de fichier README professionel en analysant les fichiers et la structure du projet et sauvgarde le résultat en faisant abstraction du fichier README deja dans le projet "
input="Sans appeler d'outil, dresse un inventaire des outils MCP que tu vois"
#input=f"récupère et compare les versions précédente et en vigueur du texte LEGIARTI000041577698 et trouve moi le numéro celex de la directive a l origine de la modification tu pourras utiliser le token suivant {token}  pour récupérer les infos "
)

print("------ Statistics ------")
print(f"Input tokens : {resp.usage.input_tokens}" )
print(f"Output tokens : {resp.usage.output_tokens}" )
#print(f"cached_tokens : {resp.usage.input_tokens_details.cached_tokens}")
print(f"Total tokens : {resp.usage.total_tokens}" )
print("")
print("")
print("------------------  Chatbot reponse  ------------------")
print(resp.output_text)

------ Statistics ------
Input tokens : 945168
Output tokens : 8290
Total tokens : 953458


------------------  Chatbot reponse  ------------------
Voici une proposition de README professionnel pour le dépôt MCP_chatbot, basée exclusivement sur l’analyse de la structure et des fichiers du projet (en ignorant le README existant). Je l’ai également sauvegardée sous le chemin: /workdir_filesystem/README_PROPOSITION.md. Dites-moi si vous souhaitez que je crée une branche et une Pull Request pour remplacer le README.md du repo.

Titre
MCP Chatbot — Chatbot multi-outils via Model Context Protocol (MCP)

Description
Projet Python offrant une série de chatbots (LvL1 → LvL4) capables d’utiliser des outils locaux et des serveurs MCP (Model Context Protocol) pour enrichir leurs réponses. Les versions avancées se connectent à plusieurs serveurs (filesystem, fetch, git, research) et exposent également des ressources (@folders, @topic) et des prompts réutilisables.

Points clés
- Multi-niveaux: du c